<div style="
    background: linear-gradient(90deg, #0f2027 0%, #2c5364 100%);
    padding: 32px 0 32px 0;
    border-radius: 15px;
    text-align: center;
    margin-bottom: 30px;
">
  <h1 style="color: #fff; letter-spacing: 2px; font-size: 2.7rem; margin: 0; font-weight: 800;">
    Building Scalable AI with Ray
  </h1>
</div>


<h2 style="color: #0f2027; background: linear-gradient(90deg, #43cea2 0%, #185a9d 100%); padding: 12px 0; border-radius: 8px; text-align:center; font-size: 2rem; letter-spacing: 1px;">
   <span style="color: #fff;">Introduction</span> 
</h2>

## What is Ray ?

- **Open-source project** under PyTorch Foundation
- **Open-source distributed scheduler** for stateless tasks & stateful actors  
- **Key features:** task graphs, resource-aware, fast data transfer, GPU/custom resources  
- **Infra:** in-memory object store, fault-tolerant design  
- **Ecosystem:** Data, Train, Tune, Serve, RLlib  
- **User-friendly Python APIs**


<div align="center"><img src="assets/img01.png" alt="Intro to Ray" width="70%"></div>

| Concept | What It Is | Why It Matters (The Problem It Solves) |
| :--- | :--- | :--- |
| **`@ray.remote`** | A decorator to mark Python code for parallel execution. | The magic switch to turn a normal function or class into a distributed building block. |
| **Task** | A remote, stateless function call. | **Problem:** My code is slow because it only uses one core. A Task lets you run a function on any available core. |
| **Actor** | A remote, stateful class instance. | **Problem:** My parallel tasks need to share and update a common state (like a counter or a model). |
| **`.remote()`** | The syntax used to execute a Task or an Actor method. | The command to "send this work to the Ray cluster" instead of running it here. |
| **`ObjectRef`** | A "future" or a "receipt" for a result being computed. | The placeholder you get back instantly after calling `.remote()`, allowing your code to continue without waiting. |
| **`ray.get()`** | The command to retrieve the actual result from an `ObjectRef`. | **Problem:** My parallel work has been sent out; now I need the final answers back. |
| **`ray.put()`** | A command to place a large object into shared memory. | **Problem:** Sending the same large dataset (e.g., a big model) to every task is slow and wasteful. |

## Ray `Task`

<div align="center"><img src="assets/img02.png" alt="Ray Task" width="70%"></div>

#### Example of a sequential process (`without Ray`)

Let's `run` it!

In [ ]:
!python code/sequential_process.py

#### Example of the same process but using `Ray Task`

Let's `run` it!

In [ ]:
!python code/parallel_process.py

## Ray `Actors`

In [ ]:
from IPython.display import Code

Code("code/ray_actor.py")

<div align="center"><img src="assets/ray_actors.jpg" alt="actors" width="80%"></div>


Let's `run` it!

In [ ]:
!python code/ray_actor.py

<h2 style="color: #0f2027; background: linear-gradient(90deg, #43cea2 0%, #185a9d 100%); padding: 12px 0; border-radius: 8px; text-align:center; font-size: 2rem; letter-spacing: 1px;">
   <span style="color: #fff;">Introduction to Ray Data</span> 
</h2>

## Streaming execution

Ray Data processes large datasets efficiently using a streaming model, which works with **blocks** as the basic units of data.

This approach replaces traditional bulk processing, where the entire dataset and intermediate results had to fit in the cluster's memory.

For example:

Here is a batch inference pipeline with a bulk processing approach.

<div align="center"><img src="assets/img04.png" alt="DP" width="80%"></div>

Note how:
- Execution is performed in stages
- The entire dataset can be repartitioned across stage boundaries

In contrast, here is the same batch inference pipeline with Ray Data's streaming model.

<div align="center"><img src="assets/img05.png" alt="DP with Ray" width="80%"></div>

Note how:
- Execution across stages of the pipeline is performed in parallel (i.e pipeline parallelism)
- Data is passed incrementally without blocking the pipeline

### Dataset and blocks

A Ray Dataset defines a data loading and processing pipeline.

When either "materialized" or "consumed", a Ray Dataset manifests as a distributed collection of blocks stored in the Ray Object Store.

Let's start by creating a materialized Ray Dataset to inspect its underlying blocks.

### Dataset and blocks

A Ray Dataset defines a data loading and processing pipeline.

When either "materialized" or "consumed", a Ray Dataset manifests as a distributed collection of blocks stored in the Ray Object Store.

Let's start by creating a materialized Ray Dataset to inspect its underlying blocks.

### `Step 1`: List the existing objects before executing the code


<div align="center"><img src="assets/img07.png" alt="DP" width="70%"></div>


In [ ]:
!ray list objects

As expected, there are no objects created yet.

### `Step 2`: Prepare some data

Let's build a parquet dataset given a target in-memory size.

In [ ]:
import gc
import ray
import numpy as np
import pandas as pd

In [ ]:

size_mb = 64

df = pd.DataFrame(
    {
        "a": np.random.rand(size_mb * 1024**2 // 8).astype(np.float64),
    }
)

memory_usage = (df.memory_usage(deep=True) / 1024**2).sum() # in MiB
print(f"Memory usage: {memory_usage} MiB")

# Write the dataframe to a parquet file 
df.to_parquet("/mnt/cluster_storage/data.parquet")

Let's inspect the parquet file.


In [ ]:
!ls -lh /mnt/cluster_storage/data.parquet

### `Step 3`: Create a materialized dataset

Let's create a `Dataset` from the parquet file using `read_parquet`

In [ ]:
ds = ray.data.read_parquet("/mnt/cluster_storage/data.parquet")
ds

Let's materialize the `Dataset` using `materialize`

In [ ]:
ds_materialized = ds.materialize() 
ds_materialized

<div align="center"><img src="assets/img09.png" alt="DP" width="70%"></div>


### `Step 4`: List the objects again

We can see an object with a size of ~64 MiB has been created. 

In [ ]:
!ray list objects

Note that we can verify the object was indeed generated by a Ray Data task by following the `CALL_SITE` of the object.

### `Step 5`: Inspect the blocks

Instead of browsing through all the objects in the object store, we can directly fetch the blocks of a materialized dataset using Ray Data.

It turns out that we can iterate over the blocks of a dataset using `iter_internal_ref_bundles`.


In [ ]:
for ref_bundle in ds_materialized.iter_internal_ref_bundles():
    print(ref_bundle)

A reference bundle `RefBundle` is simply a bundle of:
- a reference to the block
- metadata about the block

In [ ]:
block_ref, block_metadata = ref_bundle.blocks[0]
block_ref

We can check the `Block size` as well. 

Ray Data bounds block sizes to avoid excessive communication overhead and prevent out-of-memory errors.

- Small blocks are good for latency and more streamed execution
- Large blocks reduce scheduler and communication overhead.
- The default range attempts to make a good tradeoff for most jobs.

Here are the default values that Ray Data uses:

In [ ]:
max_block_size = ray.data.DatasetContext.get_current().target_max_block_size / 1024**2 # in MiB
min_block_size = ray.data.DatasetContext.get_current().target_min_block_size / 1024**2 # in MiB

print(f"Max block size: {max_block_size} MiB")
print(f"Min block size: {min_block_size} MiB")


A block is the basic unit of data that Ray Data stores in the object store and transfers over the network. 

Each block contains a disjoint subset of rows, and Ray Data loads and transforms these blocks in a distributed manner.

<div align="center"><img src="assets/img03.png" alt="Ray Data" width="80%"></div>

To fetch the block, we can use `ray.get`.

In [ ]:
block = ray.get(block_ref)
block

Ray Data stores blocks as either pandas Dataframes or pyarrow Tables. In this case, when materializing from a `read_parquet`, the block is a pyarrow Table.

<!-- TODO - figure out adding info below: -->
<!-- Note, that regardless of the data type that Ray Data uses to store the block, Ray Data will convert the block to the required batch format when batching the data and transforming it. -->

In [ ]:
type(block)

In this case, the block contains the same data as the original dataframe.

In [ ]:
block.shape, df.shape

let's clean up references to the objects we created so Ray can garbage collect them.

In [ ]:
%xdel block
%xdel block_ref
%xdel ds
%xdel ds_materialized
%xdel ref_bundle
gc.collect()

We can see that the object has been garbage collected.

In [ ]:
!ray list objects


<h2 style="color: #0f2027; background: linear-gradient(90deg, #43cea2 0%, #185a9d 100%); padding: 12px 0; border-radius: 8px; text-align:center; font-size: 2rem; letter-spacing: 1px;">
   <span style="color: #fff;">What are we going to build today ?</span> 
</h2>

* Scalable data ingestion (`Batch image generation`)
* Transforming data using Ray Data pipelines and operators
* Scalable batch inference processing with accelerators
* Joining Ray Datasets and apply data transformation to joined columns
* Integrating scalable LLM inference and fractional resource scheduling


## Scenario

We have a dataset of image prompts (in our example, animals) and another dataset with enhanced detail information for each record (in the demo, clothing the animal will wear).

Our end goal is to combine the prompts and details, then use a LLM to enhance to prompts further, employ an image gen model to create corresponding images, and produce batch output to storage.

### Ray Data motivation

It's pretty easy to write a Python script to manipulate strings and use models directly from Huggingface with code like the following:

```python
# Load some model
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16").to("cuda")
# Perform the inference
image = pipe("A cinematic shot of a racoon wearing an italian priest robe.", num_inference_steps=1, guidance_scale=0.0).images[0]
```

<div align="center"><img src="assets/img06.png" alt="RD" width="80%"></div>

But we want to build a scalable data+AI processing pipeline. To do that, we want to ...

* leverage a scale-out cluster with multiple GPUs
* read data using as much of our cluster as is useful (parallel read)
* work with the data in chunks large enough to get benefits of scale (i.e., not suffer from excessive overhead relative to the number of records)
    * but also small enough to allow for flexible scheduling as it flows through our pipeline -- we don't want an enormous chunk to hold up processing, require excessive disk or network I/O, etc.
* assign work to, e.g., CPU nodes where GPU is not required; or to smaller, cheaper GPUs where large ones are not required
* adjust batching to optimize GPU use even when ideal batch size may be different for different operations
* handle arbitrarily large datasets by leveraging a streaming execution model
* minimize I/O costs by, e.g., fusing operations where possible
* produce predictable flow by managing backpressure (i.e., ensuring data doesn't "pile up" in between pipeline stages)
* optimize via lazy execution and flexible logical + physical planners

Ray Data is designed to address these requirements, allowing us to orchestrate at scale while still straightforward Python / Huggingface code we're used to.

### Agenda and steps for incremental implementation

1. Locate our datasets in shared storage
2. Read records using Ray Data and learn how to perform basic transformations
3. Generate images across multiple GPU nodes
4. Lab activity: generate images and store all of our prompts and outputs as parquet data
5. Join animal records against clothing outfit details to build a bigger prompt and generate enhanced images
6. Lab activity: generate and export just the images as PNG files
7. Leverage a LLM to further enhance the prompts, adding seasonal content and generate images from the full pipeline
8. Lab activity: parameterize the LLM-based component so see how Ray Data supports separation of concerns
9. Wrapup

<h2 style="color: #0f2027; background: linear-gradient(90deg, #43cea2 0%, #185a9d 100%); padding: 12px 0; border-radius: 8px; text-align:center; font-size: 2rem; letter-spacing: 1px;">
   <span style="color: #fff;">Let's get started! </span> 
</h2>

In [ ]:
from diffusers import AutoPipelineForText2Image
from transformers import pipeline
from transformers.utils import logging
import numpy as np
import random
import ray
import torch
logging.set_verbosity_info()

## `Step 0`: Defining the dataset 

First, we need to get all of our data in some common location where the whole cluster can see it. This might be a blob store, NFS, database, etc.

Anyscale offers `/mnt/cluster_storage` as a NFS path.

In [ ]:
!head data/animals.csv

In [ ]:
!cp data/animals.csv /mnt/cluster_storage/

<div align="center"><img src="assets/img09.png" alt="DP" width="70%"></div>

## `Step 1`: Read the data using `Ray Data`

Ray Data's `read_xxxx` methods (see I/O in Ray Docs for all the available formats and data sources) get us scalable, parallel reads.

In [ ]:
animals = ray.data.read_csv('/mnt/cluster_storage/animals.csv')
animals.take_batch(3)

We can rename column, like this: 

In [ ]:
animals.rename_columns({'animal' : 'prompt'}) \
       .take_batch(3)

<div align="center"><img src="assets/step2.png" alt="Step2" width="70%"></div>


## `Step 2` : Generate Image 

We dont want to simply do this and process sequentially

```python
# Load some model
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16").to("cuda")
# Perform the inference
image = pipe("A cinematic shot of a racoon wearing an italian priest robe.", num_inference_steps=1, guidance_scale=0.0).images[0]
```

Stateful tranformation of datasets -- in this example, `AI inference where the state` is the image gen model (`ImageGen`) -- is done with the following pattern.

1. Define a Python class (which Ray will later instantiate across the cluster as one more actor instances to do the processing)
2. Use Dataset's `map_batches` API to tell Ray to send batches of data to the `__call__` method in the actors instances
    - `map_batches` allows us to specify resource requirements, actor pool size, batch size, and more

In [ ]:
class ImageGen():
    def __init__(self):
        self.pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16").to("cuda")
        
    def gen_image(self, prompts):
        return self.pipe(prompt=list(prompts), num_inference_steps=1, guidance_scale=0.0).images
    
    def __call__(self, batch):
        batch['image'] = self.gen_image(batch['prompt'])
        return batch

In [ ]:
animals_images = animals.repartition(2) \
                        .rename_columns({'animal' : 'prompt'}) \
                        .map_batches(ImageGen, 
                                     num_gpus=1, 
                                     compute=ray.data.ActorPoolStrategy(size=2), 
                                     batch_size=8)

<div align="center"><img src="assets/step3.png" alt="Step3" width="70%"></div>


Ray Datasets employ *lazy evaluation* for improved performance, so we can use APIs like `take_batch`, `take`, or `show` to trigger execution for development and testing purposes.

In [ ]:
examples = animals_images.take_batch(3)

In [ ]:
examples

<div align="center"><img src="assets/img10.png" alt="Step3" width="70%"></div>


In [ ]:
print(examples['prompt'][0])
examples['image'][0]

<div align="center"><img src="assets/step4.png" alt="Step4" width="70%"></div>



<div style="border: 2px solid #4F8EF7; background: #F3F8FF; border-radius: 10px; padding: 18px; margin: 18px 0;">
<h2 style="color: #25396f; margin-top: 0;">📝 <span style="color:#3971CC;">Lab:</span> Generate and Write All Output to Storage as <span style="color: #C678DD;">Parquet</span> Data</h2>

<ul style="font-size: 1.1em;">
  <li><b>Start</b> with the <span style="color: #A9A9A9;">Ray Dataset</span> you'd like to write.</li>
  <li><b>Check</b> the <a href="https://docs.ray.io/en/latest/data/api/input_output.html" target="_blank" style="color: #3971CC;">Ray Data Write API docs</a> to find a suitable <code>write</code> API.</li>
  <li><b>Remember</b> to write to a <span style="color: #228B22;">shared file location</span>, such as <code style="color: #228B22;">/mnt/cluster_storage</code>.</li>
</ul>
</div>

In [ ]:
# try your code here


<details>
    <summary style="font-size: 1.5em; color: ;"><b> Solution</b></summary>
 
 ```python
 animals_images.write_parquet('/mnt/cluster_storage/animals_images.parquet/')
 ```

<div align="center"><img src="assets/step5.png" alt="Step5" width="70%"></div>

</details>


<div align="center"><img src="assets/Part1_done.png" alt="Intro to Ray" width="70%"></div>

## `Step 3`: Improving the `prompt` using `JOIN`

<div align="center"><img src="assets/Part2.png" alt="Part2" width="70%"></div>

Ray Data supports a number of high-performance JOIN APIs.  
You can learn more here:  [Ray Data Join API Documentation](https://docs.ray.io/en/latest/data/joining-data.html)

We can use a `JOIN` to connect our `animal` records with a detailed prompt refinement unique to that record.

In [ ]:
!cp data/outfits.csv /mnt/cluster_storage/

In [ ]:
!head /mnt/cluster_storage/outfits.csv

In [ ]:
outfits = ray.data.read_csv('/mnt/cluster_storage/outfits.csv')
outfits.take_batch(3)

In [ ]:
animals_outfits = animals.join(outfits, 'inner', 1) \
                         .repartition(8)

animals_outfits.take_batch(3)

In [ ]:
# Expand the prompt to include the outfit
def expand_prompt(batch):
    batch['prompt'] = batch['animal'] + ' wearing a ' + batch['outfit']
    return batch

# Generate images for the dressed animals
dressed_animals = animals_outfits.map_batches(expand_prompt) \
                                 .map_batches(ImageGen, 
                                              batch_size=16, 
                                              compute=ray.data.ActorPoolStrategy(size=2), 
                                              num_gpus=1)

In [ ]:
# Take a look at the first 3 records
examples = dressed_animals.take_batch(3)

In [ ]:
examples

<div align="center"><img src="assets/img11.png" alt="Ray Task" width="70%"></div>

In [ ]:
print(examples['prompt'][0])
examples['image'][0]

<div align="center"><img src="assets/step4_task.png" alt="Ray Task" width="70%"></div>

<div style="border: 2px solid #4F8EF7; background: #F3F8FF; border-radius: 10px; padding: 18px; margin: 18px 0;">
<h2 style="color: #25396f; margin-top: 0;">📝 <span style="color:#3971CC;">Lab:</span> Generate Images for Input Prompts &amp; Write Them to a Folder</h2>

<ul style="font-size: 1.1em;">
  <li><b>Convert</b> the image column to NumPy arrays using <code>np.array(my_pil_image)</code> together with <code>map_batches</code>.</li>
  <li><b>Use</b> <code>dataset.write_images(...)</code> to write the NumPy image arrays to a folder of your choice (e.g., <code>/mnt/cluster_storage/generated_images/</code>).</li>
  <li><b>Check</b> the <a href="https://docs.ray.io/en/latest/data/api/input_output.html#ray.data.Dataset.write_images" target="_blank" style="color: #3971CC;">Ray Data Write Images API Docs</a> for details.</li>
</ul>
</div>

In [ ]:
# try your code here


<details>
    <summary style="font-size: 1.5em; color: ;"><b> Solution</b></summary>
 
 ```python
    def image_to_array(batch):
        batch['image'] = [np.array(i) for i in batch['image']]
        return batch
        
    dressed_animals.map_batches(image_to_array) \
                   .write_images('/mnt/cluster_storage/animals_images/', 'image')
 ```
</details>


<div align="center"><img src="assets/Part2_done.png" alt="Intro to Ray" width="70%"></div>

## `Step 4` :  Enhance pipeline with `LLM` generated prompt

We can leverage a LLM to create more varied and detailed image prompts -- as well as add dynamism like a seasonal element -- by adding a LLM batch inference step to the pipeline.

To implement this operation, we'll
1. Create a Python class to encasulate the logic and data transformtion
1. Use `map_batches` to route batches of data from our Ray Dataset through this transformation operation
1. Demonstrate Ray's support for fractional resource allocation, so that we can schedule 4 GPU-dependent operator instances with only 2 GPUs
1. Demonstrate the decoupling of operator batch sizes from each other (as well as from Dataset block size) to optimally use our models and GPUs

<div align="center"><img src="assets/Part3.png" alt="Intro to Ray" width="70%"></div>

In [ ]:
class Enhancer():
    def __init__(self):
        self.pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct", device='cuda')
        
    def chat(self, prompts):
        messages = []
        for p in prompts:
            season = random.choice(['winter', 'spring', 'summer', 'fall'])
                                   
            message = [{"role": "system", "content": "You are a helpful assistant." +
                        "Enhance the image description with two short elements corresponding to the " + season + 
                        "season. Keep animal wearing clothing and retain image medium information (like photo or painting). Return new description only, no intro."},
                        {"role": "user", "content": p }]
            messages.append(message)
        return [out[0]['generated_text'][-1]['content'] for out in self.pipe(messages, max_new_tokens=200, batch_size=2)]
    
    def __call__(self, batch):
        batch['prompt'] = self.chat(batch['prompt'])
        return batch

In [ ]:
seasonal_images = animals_outfits.map_batches(expand_prompt) \
                                 .map_batches(Enhancer, batch_size=4, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.6) \
                                 .map_batches(ImageGen, batch_size=8, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.4)

In [ ]:
examples = seasonal_images.take_batch(3)

In [ ]:
print(examples['prompt'][2])
examples['image'][2]


<div align="center"><img src="assets/Part3_done.png" alt="Intro to Ray" width="70%"></div>

<div style="border: 2px solid #4F8EF7; background: #F3F8FF; border-radius: 10px; padding: 18px; margin: 18px 0;">
<h2 style="color: #25396f; margin-top: 0;">📝 <span style="color:#3971CC;">Modify the <code>Enhancer</code> class and <code>seasonal_images</code> pipeline for parametrization</span></h2>

<ul style="font-size: 1.1em;">
  <li>Use variables contaioning the model name and the name of the dataset column containing the prompt as below</li>
</ul>
</div>

In [ ]:
enhancer_model = "Qwen/Qwen2.5-0.5B-Instruct"
prompt_column = "prompt"

In [ ]:
# try your code here: updated Enhancer class


In [ ]:
# try your code here: updated pipelineto generate seasonal_images Ray dataset


<details>
    <summary style="font-size: 1.5em; color: ;"><b> Solution</b></summary>
 
 ```python
    # try your code here: updated Enhancer class
    class Enhancer():
        def __init__(self, model_name, prompt_column_name):
            self.pipe = pipeline("text-generation", model=model_name, device='cuda')
            self.prompt_column_name = prompt_column_name
            
        def chat(self, prompts):
            messages = []
            for p in prompts:
                season = random.choice(['winter', 'spring', 'summer', 'fall'])
                                    
                message = [{"role": "system", "content": "You are a helpful assistant." +
                            "Enhance the image description with two short elements corresponding to the " + season + 
                            "season. Keep animal wearing clothing and retain image medium information (like photo or painting). Return new description only, no intro."},
                            {"role": "user", "content": p }]
                messages.append(message)
            return [out[0]['generated_text'][-1]['content'] for out in self.pipe(messages, max_new_tokens=200, batch_size=2)]
        
        def __call__(self, batch):
            batch[self.prompt_column_name] = self.chat(batch[self.prompt_column_name])
            return batch


    # try your code here: updated pipelineto generate seasonal_images Ray dataset
    seasonal_images = animals_outfits.map_batches(expand_prompt) \
                        .map_batches(Enhancer, batch_size=4, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.6, fn_constructor_args=[enhancer_model, prompt_column]) \
                        .map_batches(ImageGen, batch_size=8, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.4)
 ```
</details>

In [ ]:
# try your code here: updated Enhancer class
class Enhancer():
    def __init__(self, model_name, prompt_column_name):
        self.pipe = pipeline("text-generation", model=model_name, device='cuda')
        self.prompt_column_name = prompt_column_name
        
    def chat(self, prompts):
        messages = []
        for p in prompts:
            season = random.choice(['winter', 'spring', 'summer', 'fall'])
                                   
            message = [{"role": "system", "content": "You are a helpful assistant." +
                        "Enhance the image description with two short elements corresponding to the " + season + 
                        "season. Keep animal wearing clothing and retain image medium information (like photo or painting). Return new description only, no intro."},
                        {"role": "user", "content": p }]
            messages.append(message)
        return [out[0]['generated_text'][-1]['content'] for out in self.pipe(messages, max_new_tokens=200, batch_size=2)]
    
    def __call__(self, batch):
        batch[self.prompt_column_name] = self.chat(batch[self.prompt_column_name])
        return batch


In [ ]:
# try your code here: updated pipelineto generate seasonal_images Ray dataset
seasonal_images = animals_outfits.map_batches(expand_prompt) \
                    .map_batches(Enhancer, batch_size=4, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.6, fn_constructor_args=[enhancer_model, prompt_column]) \
                    .map_batches(ImageGen, batch_size=8, compute=ray.data.ActorPoolStrategy(size=2), num_gpus=0.4)

In [ ]:
examples = seasonal_images.take_batch(4)

In [ ]:
print(examples['prompt'][1])
examples['image'][1]

<h2 style="color: #0f2027; background: linear-gradient(90deg, #43cea2 0%, #185a9d 100%); padding: 12px 0; border-radius: 8px; text-align:center; font-size: 2rem; letter-spacing: 1px;">
   <span style="color: #fff;">Thank You!</span> 
</h2>

<div align="center"><img src="assets/LI.png" alt="Intro to Ray" width="50%"></div>